# 验证 `pulses.py` 模块

本 Notebook 用于测试和演示 `pulses.py` 模块的核心功能。我们将分步验证：
1.  基础包络类 `GaussianEnvelope` 的正确性。
2.  `DRAGCorrector` 对 `EnvelopeModulatedSequence` 的修正效果。
3.  为 RWA 和 Lab Frame 两种仿真模式生成信号的能力。

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import platform # 导入 platform 库来判断操作系统

# --- 中文显示解决方案 ---
# 根据您的操作系统，选择一个可用的中文字体
system_os = platform.system()
if system_os == 'Windows':
    # Windows 系统，例如：'SimHei' (黑体), 'Microsoft YaHei' (微软雅黑)
    plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']
elif system_os == 'Darwin':
    # macOS 系统
    plt.rcParams['font.sans-serif'] = ['PingFang SC']
else:
    # Linux 系统，需要确认已安装相应字体，例如：'WenQuanYi Micro Hei'
    plt.rcParams['font.sans-serif'] = ['WenQuanYi Micro Hei']

# 解决负号 '-' 显示为方块的问题
plt.rcParams['axes.unicode_minus'] = False
# -------------------------


# 导入我们自己库中的类
from quantum_sim.pulses import *

# 设置 matplotlib 绘图样式
# plt.style.use('seaborn-v0_8-whitegrid') # 您可以继续使用您喜欢的样式
print("模块导入及 Matplotlib 中文环境配置完成。")

## 1. 定义物理和脉冲参数

我们首先定义一套贯穿整个测试的、符合物理实际的参数。

In [ ]:
# 物理参数 (参考 Chen, 2018, Table 2.1)
qubit_freq = 5.3 * 2 * np.pi  # 目标量子比特频率 (rad/s GHz)
anharmonicity = -0.212 * 2 * np.pi      # 非谐性 (rad/s GHz)

# 脉冲参数
pulse_duration = 20.0  # ns
pulse_sigma = 5.0      # 高斯脉冲的标准差 (ns)
# 设置幅度以实现一个名义上的 π 脉冲 (面积 ≈ π)
pulse_amp = np.pi / (pulse_sigma * np.sqrt(2 * np.pi)) 

# DRAG 修正参数
drag_alpha = 0.5

# 时间轴
t_list = np.linspace(0, pulse_duration, 501)

print(f"参数设定完成。脉冲时长: {pulse_duration} ns, 非谐性: {anharmonicity/(2*np.pi):.3f} GHz")

## 2. 验证基础包络 `GaussianEnvelope`

我们创建 `GaussianEnvelope` 的实例，并可视化其包络值 `.value()` 和解析导数 `.derivative()`，以确认数学实现是否正确。

In [ ]:
# 实例化基础包络
gauss_envelope = GaussianEnvelope(
    duration=pulse_duration,
    amp=pulse_amp,
    sigma=pulse_sigma
)

# 计算包络及其导数
envelope_values = gauss_envelope.value(t_list)
derivative_values = gauss_envelope.derivative(t_list)

# 绘图
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t_list, envelope_values, 'b-', linewidth=2, label=r"包络 $\Omega(t)$")
ax.plot(t_list, derivative_values, 'g--', linewidth=2, label=r"解析导数 $\dot{\Omega}(t)$")
ax.set_title("基础包络 (GaussianEnvelope) 验证", fontsize=16)
ax.set_xlabel("时间 (ns)")
ax.set_ylabel("幅度 (rad/ns)")
ax.legend(fontsize=12)
plt.show()

In [ ]:
unmodified_sequence = MicrowaveSequence(
    envelope=gauss_envelope,
    carrier_freq=qubit_freq
)

In [ ]:
type(unmodified_sequence)

## 3. 验证 DRAG 修正与 RWA 信号生成

接下来，我们将基础包络封装成 `EnvelopeModulatedSequence`，然后使用 `DRAGCorrector` 对其进行修正，并检查修正前后的 I/Q (RWA) 信号。这是验证我们核心设计——类的组合与分离——的关键一步。

In [ ]:
# 1. 创建未修正的控制序列
unmodified_sequence = EnvelopeModulatedSequence(
    envelope=gauss_envelope,
    carrier_freq=qubit_freq
)

# 2. 创建修正器并应用
drag_corrector = DRAGCorrector(anh=anharmonicity, alpha=drag_alpha)
drag_corrected_sequence = drag_corrector.apply(unmodified_sequence)

# 3. 获取 RWA 信号
unmodified_rwa = unmodified_sequence.get_rwa_signals(t_list)
corrected_rwa = drag_corrected_sequence.get_rwa_signals(t_list)

# 4. 绘图
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(t_list, corrected_rwa['I'], 'b-', linewidth=2, label=r"修正后的 I(t) [$=\Omega(t)$]")
ax.plot(t_list, corrected_rwa['Q'], 'r-', linewidth=2, label=r"修正后的 Q(t) [$\propto \dot{\Omega}(t)$]")
ax.plot(t_list, unmodified_rwa['Q'], 'k:', label="未修正的 Q(t) [=0]")
ax.set_title("RWA 慢变包络验证 (应用 DRAG 修正)", fontsize=16)
ax.set_xlabel("时间 (ns)")
ax.set_ylabel("幅度 (rad/ns)")
ax.legend(fontsize=12)
plt.show()

## 4. 验证 Lab Frame 信号生成 (高分辨率对比)

最后，我们调用修正后与未修正序列的 `get_lab_frame_signal` 方法，并使用一个**高分辨率**的时间轴来生成最终的物理信号，以进行清晰的可视化对比。

In [ ]:
# --- 为了清晰绘制高频信号，我们创建一个分辨率更高的时间轴 ---
# 将采样点从 501 增加到 4001，确保每个振荡周期有足够的点
t_list_high_res = np.linspace(0, pulse_duration, 4001)

# 1. 获取未修正序列的 Lab Frame 信号
# S(t) = I(t)cos(ωt)，因为 Q(t)=0
unmodified_lab_signal = unmodified_sequence.get_lab_frame_signal(t_list_high_res)

# 2. 获取 DRAG 修正后序列的 Lab Frame 信号
# S(t) = I(t)cos(ωt) + Q(t)sin(ωt)
corrected_lab_signal = drag_corrected_sequence.get_lab_frame_signal(t_list_high_res)

# 3. 绘图对比
fig, axes = plt.subplots(2, 1, figsize=(6, 6))

# 绘制未修正的信号 (半透明)
axes[0].plot(t_list_high_res, unmodified_lab_signal, color='gray', alpha=0.7, linewidth=1.0, label=r"无 DRAG: $S(t) = \Omega_X(t)\cos(\omega_{10}t)$")
axes[0].set_title("未修正信号", fontsize=14)
axes[0].set_xlabel("时间 (ns)")
axes[0].set_ylabel("幅度 (rad/ns)")
axes[0].legend(fontsize=12, loc='best')
axes[0].grid(True, which='both', linestyle='--', linewidth=0.5)

# 绘制 DRAG 修正后的信号 (实体)
axes[1].plot(t_list_high_res, corrected_lab_signal, color='purple', linewidth=1.5, label=r"有 DRAG: $S(t) = \Omega_X(t)\cos(\omega_{10}t) + \Omega_Y(t)\sin(\omega_{10}t)$")
axes[1].set_title("Lab Frame 物理信号对比 (高分辨率)", fontsize=16)
axes[1].set_xlabel("时间 (ns)")
axes[1].set_ylabel("幅度 (rad/ns)")
axes[1].legend(fontsize=12, loc='best')
axes[1].grid(True, which='both', linestyle='--', linewidth=0.5)
plt.tight_layout()
plt.show()

## 5. 验证新增的 `CosineEnvelope`

现在我们来验证新添加的 `CosineEnvelope`，并展示我们的框架如何无缝地将其与 `DRAGCorrector` 结合。

In [ ]:
# --- 最终可视化验证 (三合一整合版) ---

# 1. 实例化 CosineEnvelope
cos_envelope = CosineEnvelope(
    duration=pulse_duration,
    amp=pulse_amp
)

# 2. 计算包络和导数 (使用标准时间轴)
envelope_values_cos = cos_envelope.value(t_list)
derivative_values_cos = cos_envelope.derivative(t_list)

# 3. 应用 DRAG 修正并获取 RWA 信号
cos_sequence = EnvelopeModulatedSequence(envelope=cos_envelope, carrier_freq=qubit_freq)
drag_corrected_cos_sequence = drag_corrector.apply(cos_sequence)
corrected_rwa_cos = drag_corrected_cos_sequence.get_rwa_signals(t_list)

# 4. 获取 Lab Frame 信号 (使用高分辨率时间轴以保证平滑)
t_list_high_res = np.linspace(0, pulse_duration, 4001)
lab_frame_signal_cos = drag_corrected_cos_sequence.get_lab_frame_signal(t_list_high_res)


# 5. 使用 plt.subplots(3, 1) 进行统一绘图
fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)
fig.suptitle("CosineEnvelope 功能完整验证", fontsize=18)

# -- 子图 1: 基础包络与导数 --
ax1 = axes[0]
ax1.plot(t_list, envelope_values_cos, 'b-', linewidth=2, label=r"余弦包络 $\Omega(t)$")
ax1.plot(t_list, derivative_values_cos, 'g--', linewidth=2, label=r"解析导数 $\dot{\Omega}(t)$")
ax1.set_title("1. 基础包络 (CosineEnvelope)", fontsize=14)
ax1.set_ylabel("幅度 (rad/ns)")
ax1.legend(fontsize=12)
ax1.grid(True)

# -- 子图 2: DRAG 修正后的 RWA 信号 --
ax2 = axes[1]
ax2.plot(t_list, corrected_rwa_cos['I'], 'b-', linewidth=2, label=r"修正后的 I(t)")
ax2.plot(t_list, corrected_rwa_cos['Q'], 'r-', linewidth=2, label=r"修正后的 Q(t)")
ax2.set_title("2. RWA 慢变包络 (应用 DRAG)", fontsize=14)
ax2.set_ylabel("幅度 (rad/ns)")
ax2.legend(fontsize=12)
ax2.grid(True)

# -- 子图 3: Lab Frame 物理信号 --
ax3 = axes[2]
ax3.plot(t_list_high_res, lab_frame_signal_cos, color='purple', linewidth=1.5)
ax3.set_title(r"3. Lab Frame 物理信号 $S(t)$", fontsize=14)
ax3.set_xlabel("时间 (ns)")
ax3.set_ylabel("幅度 (rad/ns)")
ax3.grid(True, which='both', linestyle='--', linewidth=0.5)

# 自动调整布局防止重叠
plt.tight_layout(rect=(0, 0.03, 1, 0.95))
plt.show()